In [ ]:
# Standard modules
import time
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np
import control
from IPython.display import display, Markdown
import ae353_spacecraft_design as design
# My own script (an interface to the simulator)
import ae353_spacecraft_simulate

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

In [ ]:
def iterate(delta):
    wheels = [
        {"alpha": 0 * np.pi / 4 + np.pi / 3, "delta": np.arctan(delta/np.sqrt(3))},
        {"alpha": 2 * np.pi / 3 + np.pi / 3, "delta": np.arctan(delta/np.sqrt(3))},
        {"alpha": 4 * np.pi / 3 + np.pi / 3, "delta": np.arctan(delta/np.sqrt(3))},
        {"alpha": 0 * np.pi / 4, "delta": -np.pi / 2},
    ]

    m, J = design.create_spacecraft(wheels)

    r = 0.25  # Adjust the radius as needed
    num_stars = 8

    # Generate star positions circularly
    stars = []
    for i in range(num_stars):
        theta = 2 * np.pi * i / num_stars  # Angle for each star
        alpha = r * np.cos(theta)          # x-coordinate (alpha)
        delta = r * np.sin(theta)          # y-coordinate (delta)
        stars.append({'alpha': alpha, 'delta': delta})

    design.create_stars(stars)

    # Define yaw, pitch, roll angles
    psi, theta, phi = sym.symbols('psi, theta, phi')

    # Define angular velocities
    w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

    # Define torques
    tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

    # Compute resultant torques
    T1 = - tau_1 * sym.Matrix(wheels[0]['xyz']) / np.linalg.norm(wheels[0]['xyz'])
    T2 = - tau_2 * sym.Matrix(wheels[1]['xyz']) / np.linalg.norm(wheels[1]['xyz'])
    T3 = - tau_3 * sym.Matrix(wheels[2]['xyz']) / np.linalg.norm(wheels[2]['xyz'])
    T4 = - tau_4 * sym.Matrix(wheels[3]['xyz']) / np.linalg.norm(wheels[3]['xyz'])
    T = sym.nsimplify(T1 + T2 + T3 + T4)

    # Define rotation matrices
    Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
    Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
    Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])

    # Define the transformation from angular velocity to angular rates
    ex = sym.Matrix([[1], [0], [0]])
    ey = sym.Matrix([[0], [1], [0]])
    ez = sym.Matrix([[0], [0], [1]])
    M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

    # Define euler's equations
    Jx, Jy, Jz = [sym.nsimplify(j) for j in np.diag(J)]
    euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                        [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                        [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

    # Define equations of motion
    f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

    f_num = sym.lambdify([psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4],f)

    #picking all eq points as zero satisfies reuquiremend of the dot terms to be zero too
    psi_e = 0.
    theta_e = 0.
    phi_e = 0.
    w_xe = 0.
    w_ye = 0.
    w_ze = 0.
    tau_1e = 0.
    tau_2e = 0.
    tau_3e = 0.
    tau_4e = 0.

    m = sym.Matrix([psi,
                    theta,
                    phi,
                    w_x,
                    w_y,
                    w_z
    ])

    n = sym.Matrix([tau_1,
                    tau_2,
                    tau_3,
                    tau_4
    ])

    A_num = sym.lambdify([psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4],f.jacobian(m))
    A = A_num(0,0,0,0,0,0,0,0,0,0)

    B_num = sym.lambdify([psi,theta,phi,w_x,w_y,w_z,tau_1,tau_2,tau_3,tau_4],f.jacobian(n))
    B = B_num(0,0,0,0,0,0,0,0,0,0)

    W = B
    shape= A.shape[0]
    for i in range(1,shape):
        col= np.linalg.matrix_power(A,i)@B
        W = np.block([[W,col]])
    W = W.astype(np.float64)
    rank = np.linalg.matrix_rank(W)
    print(shape)
    print(rank)

    #finding K, but cant find u from here because of the sensor

    import scipy.linalg
    m_e = sym.Matrix([psi_e,
                    theta_e,
                    phi_e,
                    w_xe,
                    w_ye,
                    w_ze
    ])

    n_e = sym.Matrix([tau_1e,
                    tau_2e,
                    tau_3e,
                    tau_4e
    ])

    x = m - m_e
    u = n - n_e

    Q = np.diag([2, 1, 2, 1, 3, 1])
    Q = np.diag([500, 500, 500, 1, 1, 1])
    Q_dock = np.diag([100, 100, 100, 1, 1, 1])
    R = np.diag([1,1,1,1]) * 10e-3
    R_dock = np.diag([1, 1, 1, 1])
    K, _, _ = control.lqr(A, B, Q, R)
    K_dock, _, _ = control.lqr(A, B, Q_dock, R_dock)

    eigenvalues = np.linalg.eigvals(A - B @ K)
    # Check if all eigenvalues are real (zero imaginary part) and have negative real part
    if np.all(np.isclose(np.imag(eigenvalues), 0)) and np.all(np.real(eigenvalues) < 0):
        # Only print K and the eigenvalues if all eigenvalues are real with negative real part
        print("All eigenvalues are real with negative real parts.")
        print("LQR Gain Matrix K:")
        print(K)
        print("Closed-loop Eigenvalues:")
        print(eigenvalues)
    else:
        print("Some eigenvalues have an imaginary part or positive real part.")

    alpha, delta = sym.symbols('alpha, delta')
    # Position of star in space frame
    p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                                [sym.sin(alpha) * sym.cos(delta)],
                                [sym.sin(delta)]])

    # Orientation of body frame in space frame
    R_body_in_space = Rz * Ry * Rx

    # Position of star in body frame (assuming origin of body and space frames are the same)
    p_star_in_body = R_body_in_space.T * p_star_in_space

    # Position of star in image frame
    r = sym.nsimplify(design.scope_radius)
    p_star_in_image = (1 / r) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                            [p_star_in_body[2] / p_star_in_body[0]]])

    # Sensor model for each star
    g = sym.simplify(p_star_in_image, full=True)

    # Create an empty list to store the sensor models for all stars
    sensor_models = []

    # Loop over each star in the stars list
    for star in stars:
        alpha_val = star['alpha']
        delta_val = star['delta']
        
        # Position of star in space frame
        p_star_in_space = sym.Matrix([
            [sym.cos(alpha) * sym.cos(delta)],
            [sym.sin(alpha) * sym.cos(delta)],
            [sym.sin(delta)]
        ])
        
        # Orientation of body frame in space frame
        R_body_in_space = Rz * Ry * Rx
        
        # Position of star in body frame
        p_star_in_body = R_body_in_space.T * p_star_in_space
        
        # Position of star in image frame
        r = sym.nsimplify(design.scope_radius)  # Assuming design.scope_radius is defined
        p_star_in_image = (1 / r) * sym.Matrix([
            [p_star_in_body[1] / p_star_in_body[0]],
            [p_star_in_body[2] / p_star_in_body[0]]
        ])
        
        # Sensor model for current star
        g = sym.simplify(p_star_in_image, full=True)
        
        # Substitute the star's alpha and delta values
        g_substituted = g.subs({alpha: alpha_val, delta: delta_val})
        
        # Store the model
        sensor_models.append(g_substituted)

    # sensor_models now contains the sensor models for all stars
    # Stack all individual star sensor models into one vector
        g_total = sym.Matrix.vstack(*sensor_models)
        
    g_total.subs([])  # sensor_models contains all individual g's

    g_e = g_total.subs([(phi,phi_e),(theta,theta_e),(psi,psi_e)])
    g_e = np.array(g_e).flatten()
    g_e = g_e.astype(float)

    g_num = sym.lambdify([psi,theta,phi],g_total)
    C_num = sym.lambdify([psi,theta,phi],g_total.jacobian(x))
    C = C_num(0,0,0)
    C = C.astype(float)

    #observable?

    # Initialize observability matrix with the first row being C
    W_c = C
    shape = A.shape[0]  # Number of states

    # Build the observability matrix iteratively
    for i in range(1, shape):
        row = C @ np.linalg.matrix_power(A, i)
        W_c = np.block([[W_c], [row]])  # Stack new rows

    # Convert to float for numerical stability
    W_c = W_c.astype(np.float64)

    # Compute rank
    rank_O = np.linalg.matrix_rank(W_c)

    # Print results
    print("Number of states (n):", shape)
    print("Rank of Observability Matrix:", rank_O)

    if rank_O == shape:
        print("The system is OBSERVABLE.")
    else:
        print("The system is NOT OBSERVABLE.")

    from scipy import linalg

    Q_o = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    R_o = np.diag([1, 1, 1, 1, 1, 1]) * 10e-2

    Lt, _, _ = (control.lqr(A.T, C.T, np.linalg.inv(R_o), np.linalg.inv(Q_o)))
    L = Lt.T

    print("Observer Gain Matrix L:")
    print(L)
    print(f'\np_o:\n{linalg.eigvals(A - L @ C)}')


    simulator = ae353_spacecraft_simulate.Simulator(
        display=False,
        seed=None,
    )

    m_e = sym.Matrix([psi_e,
                    theta_e,
                    phi_e,
                    w_xe,
                    w_ye,
                    w_ze
    ])

    n_e = sym.Matrix([tau_1e,
                    tau_2e,
                    tau_3e,
                    tau_4e
    ])

    class Controller:
        def __init__(self,K,L,A,B,C,psi_e,theta_e,phi_e,w_xe,w_ye,w_ze,tau_1e,tau_2e,tau_3e,tau_4e,g_e):
            # FIXME: add class variable names to this list (e.g., 'xhat')
            # if you want these variables to be logged by the simulator
            # self.variables_to_log = []
            self.K = K
            self.K_dock = K_dock
            self.L = L
            self.A = A
            self.B = B
            self.C = C
            self.psi_e = psi_e
            self.theta_e  = theta_e
            self.phi_e = phi_e
            self.w_xe = w_xe
            self.w_ye = w_ye
            self.w_ze = w_ze
            self.tau_1e = tau_1e
            self.tau_2e = tau_2e
            self.tau_3e = tau_3e
            self.tau_3e = tau_3e
            self.x = x
            self.n_e = n_e
            self.dt = 0.01
            self.g_e =  g_e
        def reset(self):
            self.xhat = np.array([0.,
                                0.,
                                0.,
                                0.,
                                0.,
                                0.])
        
        def run(self, t, star_measurements):
            """
            The variable t is the current time.

            The variable star_measurements is a 1d array of length twice the
            number N of stars:

                [y_1, z_1, y_2, z_2, ..., y_N, z_N]
            
            The image coordinates y_i and z_i of the i'th star (for i = 1, ..., N)
            are at index 2 * i - 2 and 2 * i - 1 of this array, respectively.
            """
        

            # FIXME: replace the following lines
            u = -self.K @ self.xhat
            
            # y = self.C @ self.x - star_measurements
            y = star_measurements - self.g_e
            # y = self.C @ self.x
            self.xhat += self.dt * (self.A@self.xhat + self.B @ u - self.L@(self.C@self.xhat-y))
            
            tau = u

            torque_1 = tau[0]
            torque_2 = tau[1]
            torque_3 = tau[2]
            torque_4 = tau[3]
            return torque_1, torque_2, torque_3, torque_4
    
    controller = Controller(K,L,A,B,C,psi_e,theta_e,phi_e,w_xe,w_ye,w_ze,tau_1e,tau_2e,tau_3e,tau_4e,g_e)

    def set_iterate(num_simulations):
        success_count = 0
        for i in range(num_simulations):
            simulator.reset(
                initial_conditions=None,
                scope_noise=0.1,        # <-- standard deviation of each image coordinate of each star tracker measurement
                space_debris=True,      # <-- whether or not there is space debris
                docking_time=60.        # <-- how long it takes for the space-cat to dock with the spacecraft
            )
            controller.reset()
            data = simulator.run(
                controller,           # <-- required (an instance of your Controller class)
                max_time=65.0,         # <-- optional (how long you want to run the simulation in seconds)
                data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
                video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
                print_debug=False,    # <-- optional (whether to print debug text - this is recommended when saving video)
            )
            has_docked = simulator.has_docked()
            if has_docked:
                success_count += 1
        return success_count / num_simulations

    return set_iterate(100)


In [ ]:
results = np.array([])
deltas = np.linspace(0, 3.1, 100)
for delta in deltas:
    success_rates = iterate(delta)
    results = np.append(results, success_rates)
    # Print the success rate for each delta
    print(f"Delta: {delta:.2f}, Success Rate: {success_rates:.2%}")
    # Append the success rate to the results list
    results.append(success_rates)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(results) + 1), results, marker='o')
plt.title('Success Rate of Simulations')
plt.xlabel('Simulation Number')
plt.ylabel('Success Rate')
plt.ylim(0, 1)
plt.grid()
plt.xticks(range(1, len(results) + 1))
plt.show()
# Print the success rate
print(f"Success rate: {success_rates:.2%}")